# Autonomous Agent with External Tool Integration

This notebook implements an autonomous agent capable of executing external tools. It demonstrates the "ReAct" (Reasoning + Acting) pattern where the LLM decides which tools to call based on user input. The specific example uses a notification service (Pushover) to alert a human operator when specific criteria are met.

In [ ]:
# Import dependencies
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
import gradio as gr

In [ ]:
# Initialize Environment
load_dotenv(override=True)
openai = OpenAI()

In [ ]:
# Configure Notification Service (Pushover)
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

if not pushover_user or not pushover_token:
    print("Warning: Pushover credentials not found. Notifications will be simulated.")

In [ ]:
# Define Notification Tool
def push(message):
    print(f"[Notification System] Sending: {message}")
    if pushover_user and pushover_token:
        payload = {"user": pushover_user, "token": pushover_token, "message": message}
        try:
            requests.post(pushover_url, data=payload)
        except Exception as e:
            print(f"Failed to send push notification: {e}")

In [ ]:
# Define Tool Functions

def record_user_details(email, name="Name not provided", notes="not provided"):
    """Records user interest and contact details."""
    push(f"New Lead: {name} ({email}). Notes: {notes}")
    return {"status": "success", "message": "User details recorded"}

def record_unknown_question(question):
    """Logs questions that the agent could not answer."""
    push(f"Unanswered Question: {question}")
    return {"status": "success", "message": "Question logged for review"}

In [ ]:
# Define Tool Schemas (OpenAI Format)
tools = [
    {
        "type": "function",
        "function": {
            "name": "record_user_details",
            "description": "Record a user's contact information when they express interest.",
            "parameters": {
                "type": "object",
                "properties": {
                    "email": {"type": "string", "description": "User's email address"},
                    "name": {"type": "string", "description": "User's name"},
                    "notes": {"type": "string", "description": "Contextual notes"}
                },
                "required": ["email"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "record_unknown_question",
            "description": "Log a question that the agent does not have the knowledge to answer.",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {"type": "string", "description": "The unanswered question"}
                },
                "required": ["question"]
            }
        }
    }
]

In [ ]:
# Tool Execution Handler
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Executing Tool: {tool_name}")
        
        # Dynamic function dispatch
        tool_function = globals().get(tool_name)
        if tool_function:
            result = tool_function(**arguments)
        else:
            result = {"error": f"Tool {tool_name} not found"}
            
        results.append({
            "role": "tool",
            "content": json.dumps(result),
            "tool_call_id": tool_call.id
        })
    return results

In [ ]:
# Load Agent Persona Context
name = "Ed Donner"
summary = """
Ed Donner is a CTO and AI Engineer with a background in high-frequency trading and software architecture.
He is passionate about teaching Agentic AI and building autonomous systems.
"""
linkedin_profile = """
Experience:
- Founder @ AI Education Co
- CTO @ FinTech Startup
- MD @ JP Morgan
"""

In [ ]:
# Construct System Prompt
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website. "
system_prompt += f"Your goal is to be professional and helpful. "
system_prompt += f"If you don't know an answer, use the 'record_unknown_question' tool. "
system_prompt += f"If a user wants to connect, ask for their email and use 'record_user_details'. "
system_prompt += f"\n\nContext:\n{summary}\n{linkedin_profile}"

In [ ]:
# Main Chat Loop with Tool Execution
def chat(message, history):
    # Ensure history format compatibility
    history_formatted = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_prompt}] + history_formatted + [{"role": "user", "content": message}]
    
    # Max turns to prevent infinite loops
    for _ in range(5):
        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)
        message_obj = response.choices[0].message
        
        if message_obj.tool_calls:
            # Process tool calls
            results = handle_tool_calls(message_obj.tool_calls)
            messages.append(message_obj)
            messages.extend(results)
        else:
            # Final response
            return message_obj.content
            
    return "Error: Maximum conversation turns exceeded."

In [ ]:
# Launch Interface
gr.ChatInterface(chat, type="messages").launch()